In [1]:
import MSRM

In [2]:
# 이미지 인식 모델 생성
model = MSRM.SymbolDetector(
    ['sheet_1.png', 'sheet_2.png', 'sheet_3.png', 'sheet_4.png'],
    note_confidences=0.9,   # 확률이 note_confidences가 넘는 이미지들만 추출
    note_iou_threshold=0.2  # 동일한 위치의 악상기호가 겹치는 비율이 note_iou_threshold이상이면 제거 작업
)

In [3]:
# 악상기호 인식 시작
images, datas = model.detact(preview=True)


0: 416x416 1 staff, 1 note, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 85.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 staff, 1 note, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 staff, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 12.3ms
Speed: 0.0ms preprocess, 12.3ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 11.5ms
Speed: 1.0ms preprocess, 11.5ms inferenc